<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=078a35f1d98d05b555a00a513cc7917b2e9ae8258a052d2273deec0dda08c9f6
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.0
    Uninstalling protobuf-6.32.0:
      Successfully uninstalled protobuf-6.32.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-08-29 11:11:17
-------------------
qualified stocks: 91
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  4.28 L
Current:  1.37 C
-------------------
Today PnL: 60.29 K (0.44%)
Current PnL: -20.27 L (-13.63%)
CY Booked + Current PnL: -11.23 L (-7.55%)
-------------------
Total profit:  2.72 L
Total loss:  -22.99 L
-------------------
Total Booked + Current PnL: 15.75 L (12.94%)
Total Booked PnL: 36.02 L (29.6%)
Curr Year Booked PnL: 9.04 L (6.6%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.19 C
Est FTT PnL: 81.65 L (59.61%)
Deployed:  1.22 C
Current:  1.37 C
CAGR/XIRR %: 7.51%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HINDALCO,756.01,-18.25,57.0,H-LC,7.63,112752.0,8440.0,8208.0,0.47,8.09,7.28,15.96,9.0,1.03,0.85,26.36,X5K,ATH,METALS
53,MEDANTA,1486.00,13.61,61.0,H-MC,15.07,162098.0,35898.0,10277.0,-0.18,28.45,6.34,36.59,108.0,3.49,1.22,44.58,XY24,NTT,HEALTHCARE
4,APOLLOHOSP,8285.00,-15.52,55.0,H-LC,8.40,168718.0,12541.0,13548.0,-0.76,8.03,8.03,16.71,22.0,0.93,1.27,26.87,X40N,BTT,HEALTHCARE
81,TTKPRESTIG,770.00,99.79,60.0,M-SC,8.16,86119.0,-14658.0,14752.0,1.58,-14.54,17.13,0.09,245.0,-0.99,0.65,13.14,OX40N,NTT,DURABLES
66,SBIN,863.00,-14.20,41.0,M-LC,4.25,212861.0,11382.0,15837.0,0.16,5.65,7.44,13.51,60.0,0.72,1.60,18.96,XY25,NTT,BANKS


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,RAJOOENG,143.33,-32.60,39.0,H-SC,9.98,102100.0,-400.0,41228.0,0.70,-0.39,40.38,39.83,114.0,-0.01,0.77,1.09,AR,ATH,MISC
38,ICICIGI,2260.25,-20.58,36.0,H-MC,4.91,136767.0,634.0,30485.0,-0.65,0.47,22.29,22.86,91.0,0.02,1.03,13.14,X40,ATH,INSURANCE


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,7934.00,-42.59,32.0,H-LC,13.17,240024.0,-21595.0,140798.0,-0.43,-8.25,58.66,45.57,7.0,-0.15,1.81,2.13,AR,NTT,ELECTRICAL
33,HCLTECH,1943.91,0.22,40.0,H-LC,10.19,229314.0,-12610.0,75880.0,0.74,-5.21,33.09,26.15,8.0,-0.17,1.73,8.36,X40,ATH,IT
47,JIOFIN,387.00,-0.66,42.0,H-LC,15.26,219625.0,2219.0,51282.0,1.18,1.02,23.35,24.61,48.0,0.04,1.66,56.40,XY24,BTT,FINANCE
87,VBL,671.64,-13.57,42.0,H-LC,8.07,311376.0,-4466.0,117140.0,-1.94,-1.41,37.62,35.67,5.0,-0.04,2.35,12.24,X40N,ATH,FMCG
84,UNITDSPR,1644.00,-14.66,43.0,H-LC,8.73,171706.0,-10053.0,45296.0,1.51,-5.53,26.38,19.39,86.0,-0.22,1.29,2.17,X40N,NTT,BREWERIES


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,LTIM,7230.2,-6.94,48.0,H-LC,2.93,230602.0,-15549.0,94754.0,0.03,-6.32,41.09,32.18,16.0,-0.16,1.74,27.45,X200,ATH,IT
45,ITC,452.0,-38.34,47.0,H-LC,3.02,198950.0,-1188.0,22083.0,1.48,-0.59,11.10,10.44,4.0,-0.05,1.50,4.94,X40,NTT,FMCG
55,NESTLEIND,1377.0,-12.17,51.0,H-LC,3.08,270270.0,4844.0,51946.0,-0.64,1.82,19.22,21.40,11.0,0.09,2.04,9.75,XY25,NTT,FMCG
20,CIPLA,1795.0,-18.75,61.0,H-LC,4.39,216638.0,12138.0,29268.0,0.06,5.94,13.51,20.25,10.0,0.41,1.63,15.19,X40N,BTT,PHARMA
5,ASIANPAINT,4250.0,-12.54,56.0,H-LC,7.21,220669.0,-31099.0,153321.0,0.75,-12.35,69.48,48.55,27.0,-0.20,1.66,18.70,X40,BTT,PAINTS


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,RELAXO,1176.00,-33.88,58.0,H-SC,8.51,61402.0,-54136.0,89125.0,7.15,-46.86,145.15,30.28,136.0,-0.61,0.46,20.72,X40N,NTT,FOOTWEAR
64,SAMMAANCAP,326.00,-164.77,53.0,M-SC,8.24,74610.0,-27600.0,120988.0,5.41,-27.00,162.16,91.37,208.0,-0.23,0.56,21.12,XY25,NTT,FINANCE
23,COLPAL,3767.14,-0.59,56.0,H-MC,10.63,181194.0,-30547.0,112648.0,2.66,-14.43,62.17,38.77,84.0,-0.27,1.37,7.85,XY25,ATH,FMCG
68,SHALBY,327.00,890.43,44.0,M-SC,4.79,134788.0,-46355.0,91818.0,2.38,-25.59,68.12,25.10,235.0,-0.50,1.02,8.02,XY24,NTT,HEALTHCARE
22,COFFEEDAY,80.00,-43.66,65.0,L-SC,26.26,83520.0,-30029.0,70082.0,2.26,-26.45,83.91,35.27,268.0,-0.43,0.63,104.42,XR,NTT,HOTELS


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,VBL,671.64,-13.57,42.0,H-LC,8.07,311376.0,-4466.0,117140.0,-1.94,-1.41,37.62,35.67,5.0,-0.04,2.35,12.24,X40N,ATH,FMCG
67,SFL,1287.00,34.91,50.0,M-SC,9.44,178020.0,-84198.0,154023.0,-1.07,-32.11,86.52,26.63,229.0,-0.55,1.34,10.68,XY24,NTT,MISC
43,INFY,2275.00,-18.88,46.0,H-LC,11.71,316369.0,3481.0,168213.0,-0.99,1.11,53.17,54.87,3.0,0.02,2.38,7.84,X40,BTT,IT
37,HONAUT,58357.33,-17.70,55.0,H-SC,3.23,115950.0,-11988.0,59123.0,-0.94,-9.37,50.99,36.84,143.0,-0.20,0.87,18.70,X40N,ATH,ELECTRICAL
4,APOLLOHOSP,8285.00,-15.52,55.0,H-LC,8.40,168718.0,12541.0,13548.0,-0.76,8.03,8.03,16.71,22.0,0.93,1.27,26.87,X40N,BTT,HEALTHCARE


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,TTKPRESTIG,770.0,99.79,60.0,M-SC,8.16,86119.0,-14658.0,14752.0,1.58,-14.54,17.13,0.09,245.0,-0.99,0.65,13.14,OX40N,NTT,DURABLES
82,UJJIVANSFB,60.0,82.03,41.0,H-SC,14.60,115128.0,-27351.0,46869.0,0.49,-19.20,40.71,13.70,163.0,-0.58,0.87,35.88,OX40N,NTT,BANKS
48,KANSAINER,340.0,-68.52,52.0,H-SC,3.43,217602.0,-52065.0,88390.0,1.57,-19.31,40.62,13.47,138.0,-0.59,1.64,10.06,XY24,NTT,PAINTS
19,CERA,9475.0,-21.00,38.0,H-SC,3.05,112554.0,-31249.0,57999.0,0.10,-21.73,51.53,18.60,149.0,-0.54,0.85,24.21,OX40N,NTT,CERAMICS
41,INDIGOPNTS,1408.0,107.03,38.0,M-SC,2.21,135892.0,-38667.0,38702.0,-0.13,-22.15,28.48,0.02,221.0,-1.00,1.02,18.20,OX40N,NTT,PAINTS


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TCS,4998.00,-26.76,49.0,H-LC,15.58,291682.0,-54274.0,178130.0,0.30,-15.69,61.07,35.80,1.0,-0.30,2.20,3.33,X40,BTT,IT
43,INFY,2275.00,-18.88,46.0,H-LC,11.71,316369.0,3481.0,168213.0,-0.99,1.11,53.17,54.87,3.0,0.02,2.38,7.84,X40,BTT,IT
45,ITC,452.00,-38.34,47.0,H-LC,3.02,198950.0,-1188.0,22083.0,1.48,-0.59,11.10,10.44,4.0,-0.05,1.50,4.94,X40,NTT,FMCG
87,VBL,671.64,-13.57,42.0,H-LC,8.07,311376.0,-4466.0,117140.0,-1.94,-1.41,37.62,35.67,5.0,-0.04,2.35,12.24,X40N,ATH,FMCG
1,ABB,7934.00,-42.59,32.0,H-LC,13.17,240024.0,-21595.0,140798.0,-0.43,-8.25,58.66,45.57,7.0,-0.15,1.81,2.13,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,MEDANTA,1486.0,13.61,61.0,H-MC,15.07,162098.0,35898.0,10277.0,-0.18,28.45,6.34,36.59,108.0,3.49,1.22,44.58,XY24,NTT,HEALTHCARE
27,DMART,5201.0,-7.84,71.0,H-LC,12.79,230104.0,45623.0,24736.0,-0.08,24.73,10.75,38.14,38.0,1.84,1.73,37.96,X40N,NTT,FMCG


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,MEDANTA,1486.00,13.61,61.0,H-MC,15.07,162098.0,35898.0,10277.0,-0.18,28.45,6.34,36.59,108.0,3.49,1.22,44.58,XY24,NTT,HEALTHCARE
14,BLUESTARCO,2326.38,9.94,60.0,H-MC,4.04,189710.0,25040.0,42931.0,0.90,15.21,22.63,41.28,89.0,0.58,1.43,24.40,X40N,ATH,AC
88,VOLTAS,1918.49,-1.78,59.0,H-MC,6.34,205950.0,14208.0,81824.0,1.01,7.41,39.73,50.08,99.0,0.17,1.55,14.71,XY25,ATH,AC
34,HEROMOTOCO,5949.07,-3.95,70.0,H-MC,15.86,178098.0,27185.0,30116.0,-0.06,18.01,16.91,37.97,93.0,0.90,1.34,47.88,AR,ATH,AUTO
24,DABUR,735.00,-7.26,52.0,H-MC,3.74,201825.0,4797.0,84827.0,0.71,2.43,42.03,45.49,102.0,0.06,1.52,15.79,XY24,BTT,FMCG


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
34,HEROMOTOCO,5949.07,-3.95,70.0,H-MC,15.86,178098.0,27185.0,30116.0,-0.06,18.01,16.91,37.97,93.0,0.90,1.34,47.88,AR,ATH,AUTO
40,INDIAMART,4850.92,-50.72,51.0,H-SC,12.48,137646.0,14310.0,119449.0,0.75,11.60,86.78,108.45,119.0,0.12,1.04,36.55,AR,ATH,MISC
89,WHIRLPOOL,2270.00,-41.93,50.0,M-SC,5.61,98152.0,6654.0,72093.0,0.93,7.27,73.45,86.07,223.0,0.09,0.74,42.54,XR,NTT,DURABLES
90,WIPRO,420.00,-11.52,53.0,M-LC,7.70,155694.0,4749.0,104704.0,0.19,3.15,67.25,72.51,53.0,0.05,1.17,9.36,XR,NTT,IT
59,RAJOOENG,143.33,-32.60,39.0,H-SC,9.98,102100.0,-400.0,41228.0,0.70,-0.39,40.38,39.83,114.0,-0.01,0.77,1.09,AR,ATH,MISC


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.00,-43.66,65.0,L-SC,26.26,83520.0,-30029.0,70082.0,2.26,-26.45,83.91,35.27,268.0,-0.43,0.63,104.42,XR,NTT,HOTELS
3,ALKYLAMINE,4546.37,-6.52,36.0,H-SC,16.51,87028.0,-13935.0,108463.0,0.01,-13.80,124.63,93.63,148.0,-0.13,0.66,29.63,SR,ATH,CHEMICALS
52,MASFIN,398.61,-19.60,45.0,H-SC,12.70,91500.0,-6480.0,28081.0,-0.51,-6.61,30.69,22.05,152.0,-0.23,0.69,33.00,XR,ATH,FINANCE
35,HINDALCO,756.01,-18.25,57.0,H-LC,7.63,112752.0,8440.0,8208.0,0.47,8.09,7.28,15.96,9.0,1.03,0.85,26.36,X5K,ATH,METALS
82,UJJIVANSFB,60.00,82.03,41.0,H-SC,14.60,115128.0,-27351.0,46869.0,0.49,-19.20,40.71,13.70,163.0,-0.58,0.87,35.88,OX40N,NTT,BANKS


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.00,-43.66,65.0,L-SC,26.26,83520.0,-30029.0,70082.0,2.26,-26.45,83.91,35.27,268.0,-0.43,0.63,104.42,XR,NTT,HOTELS
34,HEROMOTOCO,5949.07,-3.95,70.0,H-MC,15.86,178098.0,27185.0,30116.0,-0.06,18.01,16.91,37.97,93.0,0.90,1.34,47.88,AR,ATH,AUTO
27,DMART,5201.00,-7.84,71.0,H-LC,12.79,230104.0,45623.0,24736.0,-0.08,24.73,10.75,38.14,38.0,1.84,1.73,37.96,X40N,NTT,FMCG
14,BLUESTARCO,2326.38,9.94,60.0,H-MC,4.04,189710.0,25040.0,42931.0,0.90,15.21,22.63,41.28,89.0,0.58,1.43,24.40,X40N,ATH,AC
72,STARHEALTH,761.00,10.24,60.0,H-SC,2.57,163822.0,-36394.0,112415.0,1.93,-18.18,68.62,37.97,144.0,-0.32,1.23,31.54,XY24,NTT,INSURANCE


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.63
1,25,43.07
2,50,72.69


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    37.04
LC    36.73
MC    26.19
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.18
X40      15.50
X40N     14.22
XY25      9.75
AR        9.32
XR        8.94
OX40N     7.31
X5K       2.24
X200      1.74
MH        1.67
SR        1.09
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    28.58
H-MC    23.32
H-SC    22.37
M-SC    13.13
M-LC     7.10
M-MC     2.55
L-SC     1.54
L-LC     1.05
L-MC     0.32
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.14,-1.75,34.89
IT,13.39,-16.18,73.80
BANKS,7.34,-16.98,66.65
FINANCE,7.23,-25.72,72.07
PAINTS,5.94,-17.07,43.27
MISC,5.83,-18.79,86.89
ELECTRICAL,5.64,-9.89,70.46
AUTO,4.54,-12.10,60.87
AC,3.71,1.76,31.51


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2895610.0,23
AR,1242958.0,10
XR,1106368.0,13
X40,909902.0,10
X40N,544424.0,11
OX40N,488924.0,10
XY25,447748.0,7
SR,199997.0,2
X5K,140774.0,2


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2806800.0,27
M-SC,1500928.0,17
H-LC,1439094.0,17
H-MC,1282069.0,18
M-LC,435240.0,5
M-MC,339536.0,2
L-SC,256795.0,3
L-MC,59911.0,1
L-LC,44614.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,944002.0,6
M-SC,XY24,881618.0,7
H-SC,AR,667010.0,5
H-LC,X40,597627.0,5
H-SC,XR,532256.0,6
H-MC,XY24,497039.0,5
H-LC,AR,385398.0,2
M-MC,XY24,339536.0,2
H-LC,X40N,229988.0,5


In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 32.0 seconds
